## Algoritmo para predecir el LTV a 24 meses

### Preparación de los datos

1. **Colección y limpieza de datos:**
    - Asegurarse de que los datos incluyan información relevante como demografía del cliente, historial de compras y métricas de interacción.
    - Limpiar los datos para manejar valores faltantes, outliers y consistencia de formatos.
    - Define y calcula la variable objetivo.

### Ingeniería de características

2. **Creación de características:**
    - Crear características que reflejen el comportamiento del cliente, tales como el gasto total, la cantidad promedio de transacciones, la frecuencia de compra, y el tiempo desde la última compra.
    - Considerar la creación de características en ventanas de tiempo como el gasto total en los últimos 6 o 12 meses.

### Análisis exploratorio de datos (EDA)

3. **Análisis exploratorio:**
    - Analizar los datos para entender patrones y relaciones.
    - Visualizar las distribuciones del gasto y otras métricas relevantes a través de diferentes segmentos de clientes.
    - Agrupar clientes basándose en la fecha de su primera compra y seguir su comportamiento a lo largo del tiempo.


### Modelado predictivo

4. **Modelado predictivo:**
    - Dividir los datos en conjuntos de entrenamiento y prueba.
    - Elegir un modelo de regresión adecuado para predecir un resultado continuo como el LTV.
    - Entrenar el modelo en los datos de entrenamiento utilizando las características desarrolladas.

### Evaluación del modelo

5. **Evaluación del modelo:**
    - Evaluar el modelo en el conjunto de prueba usando métricas adecuadas, como el error cuadrático medio (RMSE) o el error absoluto medio (MAE).
    - Analizar los residuales para asegurar que el modelo funciona bien en todos los segmentos de clientes.

### Interpretación del modelo y aplicación de insights

6. **Interpretación y acción:**
    - Interpretar el modelo para entender qué características son más predictivas del LTV.
    - Usar la salida del modelo para segmentar clientes basándose en su LTV predicho y dirigirles estrategias de marketing específicas.

### Despliegue y monitoreo

7. **Despliegue y monitoreo:**
    - Implementar el modelo en un entorno de producción donde pueda predecir el LTV de nuevos clientes.
    - Monitorear regularmente el desempeño del modelo y actualizarlo según sea necesario para manejar cambios en el comportamiento del cliente o en las condiciones del mercado.

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
df_data = pd.read_csv('data/tlacuachitos_vip_customers_data.csv')
df_data.head()

,CustomerID,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort
0,1,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31
1,2,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31
2,3,46,57978.753383,3,Bachelor,Finance,Europe,1,2019-05-31
3,4,32,60445.266900,3,High School,Education,South America,1,2021-02-28
4,5,60,57741.870929,5,Bachelor,Entertainment,Asia,0,2018-10-31


In [3]:
df_transactions = pd.read_csv('data/tlacuachitos_vip_transactions.csv')
df_transactions.head()

,CustomerID,TransactionDate,TransactionAmount
0,1,2023-10-31,518.444092
1,1,2024-07-31,353.796197
2,1,2024-01-31,38.206591
3,1,2024-06-30,724.929423
4,2,2022-02-28,145.616000


In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           1143 non-null   int64  
 1   Age                  1143 non-null   int64  
 2   Income               1143 non-null   float64
 3   Tenure               1143 non-null   int64  
 4   Education            1143 non-null   object 
 5   Industry             1143 non-null   object 
 6   Geographic Location  1143 non-null   object 
 7   Churn_Risk           1143 non-null   int64  
 8   Cohort               1143 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 80.5+ KB


In [5]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346 entries, 0 to 4345
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         4346 non-null   int64  
 1   TransactionDate    4346 non-null   object 
 2   TransactionAmount  4346 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 102.0+ KB


In [6]:
df_transactions['TransactionDate'] = pd.to_datetime(df_transactions['TransactionDate'])
df_data['Cohort'] = pd.to_datetime(df_data['Cohort'])

In [7]:
snapshot_date = df_transactions['TransactionDate'].max()
snapshot_date

Timestamp('2024-08-31 00:00:00')

In [8]:
df_transactions['TransactionDate'].min()

Timestamp('2018-01-31 00:00:00')

In [9]:
df_data['customer_tenure'] = (snapshot_date.year - df_data['Cohort'].dt.year)*12 + (
    snapshot_date.month - df_data['Cohort'].dt.month)

df_data.head()

,CustomerID,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort,customer_tenure
0,1,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
1,2,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31,36
2,3,46,57978.753383,3,Bachelor,Finance,Europe,1,2019-05-31,63
3,4,32,60445.266900,3,High School,Education,South America,1,2021-02-28,42
4,5,60,57741.870929,5,Bachelor,Entertainment,Asia,0,2018-10-31,70


In [10]:
df_master = df_transactions.merge(df_data, on='CustomerID')
df_master.head()

,CustomerID,TransactionDate,TransactionAmount,Age,Income,Tenure,Education,Industry,Geographic Location,Churn_Risk,Cohort,customer_tenure
0,1,2023-10-31,518.444092,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
1,1,2024-07-31,353.796197,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
2,1,2024-01-31,38.206591,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
3,1,2024-06-30,724.929423,56,52752.677346,3,Master,Technology,Europe,1,2023-08-31,12
4,2,2022-02-28,145.616000,69,55297.364348,6,Bachelor,Technology,South America,0,2021-08-31,36


In [11]:
df_master['customer_tenure_on_transaction'] = (
    df_master['TransactionDate'].dt.year - df_master['Cohort'].dt.year)*12 + (
    df_master['TransactionDate'].dt.month - df_master['Cohort'].dt.month
)
df_master['customer_tenure_on_transaction']

0        2
1       11
2        5
3       10
4        6
        ..
4341     7
4342     2
4343     7
4344    10
4345    17
Name: customer_tenure_on_transaction, Length: 4346, dtype: int64

In [12]:
cltv_24_months = df_master[
    (df_master['customer_tenure'] > 24) &
    (df_master['customer_tenure_on_transaction'] <= 24)
].groupby('CustomerID')['TransactionAmount'].sum().reset_index()

cltv_24_months

,CustomerID,TransactionAmount
0,2,145.616000
1,3,487.285458
2,4,1095.025887
3,5,1685.260152
4,6,728.364381
...,...,...
857,1136,189.961932
858,1138,2375.255820
859,1139,1327.509070
860,1141,718.548745


In [13]:
categorical_features = ['Education', 'Industry', 'Geographic Location']
numerical_features = ['Age', 'Income', 'Tenure']

data_encoded = pd.get_dummies(df_data[['CustomerID']+categorical_features], 
                              columns=categorical_features, 
                              drop_first=False)



df_data_with_encoded = df_data[['CustomerID']+numerical_features].merge(
    data_encoded, on='CustomerID')

df_to_model = cltv_24_months.merge(df_data_with_encoded, on='CustomerID')

df_to_model.rename(columns={'TransactionAmount': 'LTV'}, inplace=True)

df_to_model

,CustomerID,LTV,Age,Income,Tenure,Education_Bachelor,Education_High School,Education_Master,Education_PhD,Industry_Education,Industry_Entertainment,Industry_Finance,Industry_Healthcare,Industry_Technology,Geographic Location_Asia,Geographic Location_Australia,Geographic Location_Europe,Geographic Location_North America,Geographic Location_South America
0,2,145.616000,69,55297.364348,6,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,3,487.285458,46,57978.753383,3,1,0,0,0,0,0,1,0,0,0,0,1,0,0
2,4,1095.025887,32,60445.266900,3,0,1,0,0,1,0,0,0,0,0,0,0,0,1
3,5,1685.260152,60,57741.870929,5,1,0,0,0,0,1,0,0,0,1,0,0,0,0
4,6,728.364381,25,57132.404622,3,0,0,1,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,1136,189.961932,67,59279.959062,4,0,1,0,0,0,0,1,0,0,1,0,0,0,0
858,1138,2375.255820,28,59996.754751,5,1,0,0,0,0,1,0,0,0,0,0,0,1,0
859,1139,1327.509070,26,45236.402018,8,1,0,0,0,1,0,0,0,0,0,0,1,0,0
860,1141,718.548745,29,57043.387540,5,0,1,0,0,0,0,1,0,0,0,0,0,1,0


In [14]:
import statsmodels.api as sm

In [15]:
x = ['Tenure', 'Education_Bachelor',
     'Education_Master', 'Education_PhD', 'Industry_Education',
     'Industry_Entertainment', 'Industry_Technology', 'Geographic Location_North America']
y = ['LTV']

In [16]:
X = sm.add_constant(df_to_model[x])

model = sm.OLS(df_to_model[y], X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    LTV   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     14.78
Date:                Thu, 10 Oct 2024   Prob (F-statistic):           2.21e-20
Time:                        18:00:29   Log-Likelihood:                -6771.0
No. Observations:                 862   AIC:                         1.356e+04
Df Residuals:                     853   BIC:                         1.360e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [20]:
new_customer_data = {
    'const': [1], 
    'Tenure': [48],
    'Education_Bachelor': [0],
    'Education_Master': [1], 
    'Education_PhD': [0],
    'Industry_Education': [0],
    'Industry_Entertainment': [0],
    'Industry_Technology': [1],
    'Geographic Location_North America': [0]
}

new_customer_df = pd.DataFrame(new_customer_data)

predicted_ltv = results.predict(new_customer_df)

print(f"Predicted LTV for the new customer: ${predicted_ltv[0]:,.2f}")


Predicted LTV for the new customer: $3,560.52


# Resumen del Modelo OLS de Predicción de LTV

## Evaluación del Poder Predictivo y Significancia Estadística

### 1. **R-cuadrado (R²) y R-cuadrado Ajustado (Adj. R²)**
- **R² = 0.122**: Esto indica que el **12.2%** de la varianza en el valor de vida del cliente (LTV) es explicada por las variables independientes (Tenencia, Educación, Industria, Ubicación geográfica, etc.). Este R² es relativamente bajo, lo que sugiere que el modelo no está explicando una gran parte de la variabilidad en el LTV.
- **R² ajustado = 0.114**: Ajustado por el número de predictores, el R² ajustado es ligeramente más bajo, lo que indica que añadir más variables no mejora significativamente el modelo.

**Conclusión**: El bajo R² indica que el modelo tiene un poder predictivo limitado y que una parte importante de la variabilidad en el LTV no está siendo explicada por las variables incluidas.

---

### 2. **Estadístico F y Probabilidad (p-valor del F-estadístico)**
- **Estadístico F = 14.78**: Este valor prueba la significancia conjunta de todos los coeficientes de regresión.
- **p-valor del F-estadístico = 2.21e-20**: El p-valor es extremadamente bajo, lo que indica que el modelo es **estadísticamente significativo**. Al menos una de las variables predictoras tiene un impacto relevante en el LTV.

**Conclusión**: A pesar del bajo R², el modelo en su conjunto es estadísticamente significativo, lo que significa que las variables incluidas no son irrelevantes.

---

### 3. **Log-Likelihood, AIC y BIC**
- **Log-Likelihood = -6771.0**: No se interpreta directamente, pero se usa en el cálculo de AIC y BIC.
- **AIC = 13560** y **BIC = 13600**: Estos criterios de información permiten comparar modelos. Un valor más bajo indica un mejor ajuste del modelo, considerando también la penalización por el número de variables. Dado que no estamos comparando otros modelos, estos valores reflejan el equilibrio entre la calidad del ajuste y la complejidad del modelo.

---

### 4. **Grados de Libertad**
- **Df Model = 8**: El número de variables independientes utilizadas en el modelo.
- **Df Residuals = 853**: El número de observaciones menos el número de parámetros estimados, lo que refleja los grados de libertad restantes.

---

## Conclusión General

1. **Poder Predictivo**:
   - El modelo tiene un **poder predictivo bajo**, con un R² de 0.122. Esto significa que las variables incluidas (tenencia, nivel educativo, industria y ubicación geográfica) explican solo una pequeña parte de la variabilidad en el valor de vida del cliente (LTV). Es probable que haya otros factores importantes que no se han considerado.

2. **Significancia Estadística**:
   - Aunque el poder predictivo es limitado, el modelo es **estadísticamente significativo**. El p-valor extremadamente bajo del F-estadístico indica que las variables en conjunto tienen un impacto relevante sobre el LTV.

3. **Pasos Siguientes**:
   - **Incorporar más variables**: El bajo R² sugiere que es necesario añadir más variables que puedan influir en el LTV, como datos de comportamiento del cliente, hábitos de gasto, interacciones con productos, etc.
   - **Considerar modelos no lineales**: La baja capacidad explicativa del modelo podría indicar que las relaciones entre las variables y el LTV no son lineales. Modelos como árboles de decisión o técnicas de aprendizaje automático pueden mejorar la capacidad predictiva.
   - **Verificar multicolinealidad e interacciones**: Analizar posibles interacciones entre las variables o verificar si existe multicolinealidad podría ayudar a mejorar la precisión del modelo.

En resumen, el modelo actual proporciona una base estadísticamente significativa, pero con un margen amplio para mejorar su capacidad predictiva.
